In [7]:
import pandas as pd
import numpy as np

In [8]:
# Fungsi Keanggotaan Umum
def fungsi_segitiga(x, a, b, c):
    if x <= a or x >= c:
        return 0
    elif a < x < b:
        return (x - a) / (b - a)
    elif b <= x < c:
        return (c - x) / (c - b)
    else:
        return 0


def fungsi_trapesium(x, a, b, c, d):
    if x <= a or x >= d:
        return 0
    elif a < x < b:
        return (x - a) / (b - a) if b != a else 1
    elif b <= x <= c:
        return 1
    elif c < x < d:
        return (d - x) / (d - c) if d != c else 1
    else:
        return 0

In [9]:
# Fuzifikasi untuk setiap variabel input
def fuzzify_age(age_val):
    return {
        'Muda': fungsi_trapesium(age_val, 0, 0, 30, 40),
        'ParuhBaya': fungsi_segitiga(age_val, 30, 45, 60),
        'Tua': fungsi_trapesium(age_val, 50, 65, 100, 100)
    }

def fuzzify_glucose(glucose_val):
    return {
        'Normal': fungsi_trapesium(glucose_val, 50, 50, 100, 120),
        'Tinggi': fungsi_trapesium(glucose_val, 100, 140, 280, 280)
    }

def fuzzify_bmi(bmi_val):
    return {
        'Normal': fungsi_trapesium(bmi_val, 18, 18, 25, 30),
        'Berlebih': fungsi_trapesium(bmi_val, 25, 30, 50, 50)
    }

In [10]:
# Output Linguistik dan Fungsi Keanggotaan Output
def fuzzify_output(x):
    return {
        'Rendah': fungsi_trapesium(x, 0, 0, 0.2, 0.4),
        'Sedang': fungsi_segitiga(x, 0.3, 0.5, 0.7),
        'Tinggi': fungsi_trapesium(x, 0.6, 0.8, 1, 1)
    }

In [ ]:
# Aturan Fuzzy Mamdani
def apply_rules_mamdani(age_fuzzy, glucose_fuzzy, bmi_fuzzy):
    rules = [
        ('Tua', 'Tinggi', 'Berlebih', 'Tinggi'),
        ('Tua', 'Tinggi', 'Normal', 'Tinggi'),
        ('Tua', 'Normal', 'Berlebih', 'Tinggi'),
        ('ParuhBaya', 'Tinggi', 'Berlebih', 'Tinggi'),
        ('ParuhBaya', 'Tinggi', 'Normal', 'Sedang'),
        ('ParuhBaya', 'Normal', 'Berlebih', 'Sedang'),
        ('Tua', 'Normal', 'Normal', 'Sedang'),
        ('Muda', 'Tinggi', 'Berlebih', 'Sedang'),
        ('Muda', 'Tinggi', 'Normal', 'Sedang'),
        ('Muda', 'Normal', 'Berlebih', 'Sedang'),
        ('ParuhBaya', 'Normal', 'Normal', 'Rendah'),
        ('Muda', 'Normal', 'Normal', 'Rendah'),
    ]
    output_membership = {'Rendah': [], 'Sedang': [], 'Tinggi': []}
    for age_label, glucose_label, bmi_label, out_label in rules:
        w = min(age_fuzzy[age_label], glucose_fuzzy[glucose_label], bmi_fuzzy[bmi_label])
        output_membership[out_label].append(w)
    return {k: max(v) if v else 0 for k, v in output_membership.items()}

In [12]:
# Defuzzifikasi Mamdani
def defuzzify_mamdani(output_membership):
    x = np.linspace(0, 1, 101)
    aggregated = np.zeros_like(x)
    for label, degree in output_membership.items():
        mf = np.array([fuzzify_output(xi)[label] for xi in x])
        aggregated = np.maximum(aggregated, np.minimum(degree, mf))
    if aggregated.sum() == 0:
        return 0
    return np.sum(x * aggregated) / np.sum(aggregated)

In [13]:
# Prediksi satu baris
def predict_stroke_risk_mamdani(input_data):
    age_fuzzy = fuzzify_age(input_data['age'])
    glucose_fuzzy = fuzzify_glucose(input_data['avg_glucose_level'])
    bmi_fuzzy = fuzzify_bmi(input_data['bmi'])
    output_membership = apply_rules_mamdani(age_fuzzy, glucose_fuzzy, bmi_fuzzy)
    return defuzzify_mamdani(output_membership)

In [14]:
# Load dataset yang telah diproses
df = pd.read_csv('stroke_preprocessed_for_fuzzy.csv')

In [15]:
def predict_row(row):
    input_data = {
        'age': row['age'],
        'avg_glucose_level': row['avg_glucose_level'],
        'bmi': row['bmi']
    }
    return predict_stroke_risk_mamdani(input_data)

In [16]:
df['mamdani_risk_score'] = df.apply(predict_row, axis=1)
df['mamdani_pred'] = (df['mamdani_risk_score'] >= 0.5).astype(int)

In [17]:
# Evaluasi
y_true = df['stroke']
y_pred = df['mamdani_pred']

In [18]:
# Hitung akurasi dan F1-score manual
def accuracy(y_true, y_pred):
    return np.mean(np.array(y_true) == np.array(y_pred))

def f1_score_manual(y_true, y_pred):
    tp = sum((y_true == 1) & (y_pred == 1))
    fp = sum((y_true == 0) & (y_pred == 1))
    fn = sum((y_true == 1) & (y_pred == 0))
    if tp + fp == 0 or tp + fn == 0:
        return 0
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if precision + recall == 0:
        return 0
    return 2 * precision * recall / (precision + recall)

acc = accuracy(y_true, y_pred)
f1 = f1_score_manual(np.array(y_true), np.array(y_pred))

print("=== Evaluasi Fuzzy Mamdani ===")
print(f"Akurasi  : {acc:.4f}")
print(f"F1-score : {f1:.4f}")

=== Evaluasi Fuzzy Mamdani ===
Akurasi  : 0.5255
F1-score : 0.1459
